In [164]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [165]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [166]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [167]:
%%capture pwd
!pwd

In [168]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-are-the-likely-costs-of-the-transition-to-a-sustainable-economy',
 300,
 500)

# Fig 1

In [169]:
# https://www.theccc.org.uk/wp-content/uploads/2020/06/Reducing-UK-emissions-Progress-Report-to-Parliament-Committee-on-Cli.._-002-1.pdf
# https://www.irena.org/publications/2020/Jun/Renewable-Power-Costs-in-2019
df1 = pd.read_excel(
    "raw/200605-IRENADatafileRenewablePowerGenerationCostsin2019v11.xlsx",
    sheet_name="Figures ES.2 & 1.3",
    skiprows=7,
    nrows=30,
    usecols="B:L",
).dropna(how="any")
df1["Unnamed: 1"] = [
    i + "_" + j
    for i in ["pv", "onwind", "offwind", "csp"]
    for j in ["p5", "avg", "p95"]
]
df2 = pd.read_excel(
    "raw/200605-IRENADatafileRenewablePowerGenerationCostsin2019v11.xlsx",
    sheet_name="Figures ES.2 & 1.3",
    skiprows=7,
    nrows=30,
    usecols="M:Z",
).dropna(how="all")
df2["2010.1"] = df2["2010.1"].replace("Auctions and PPA database", np.nan)
df2.columns = range(2010, 2024)
df2 = df2.dropna(subset=[2020])
df2 = pd.concat([df1["Unnamed: 1"], df2], axis=1)
df2 = df2.dropna(subset=["Unnamed: 1"], axis=0)
df1["db"] = "lcoe"
df2["db"] = "auction"
df = pd.concat([df1, df2])
df["tech"] = df["Unnamed: 1"].astype(str).str.split("_").str[0]
df["conf"] = df["Unnamed: 1"].astype(str).str.split("_").str[1]
df = (
    df.drop("Unnamed: 1", axis=1)
    .set_index(["db", "tech", "conf"])
    .stack()
    .reset_index()
    .set_index(["level_3", "tech", "db", "conf"])
    .unstack()[0]
    .reset_index()
)
df.columns = ["year", "tech", "db", "avg", "p5", "p95"]

In [170]:
f = "fig1_lcoe"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,tech,db,avg,p5,p95
0,2010,csp,lcoe,0.346,0.276,0.39
1,2010,offwind,lcoe,0.161,0.114,0.204
2,2010,onwind,auction,0.082,0.054,0.105
3,2010,onwind,lcoe,0.086,0.058,0.117
4,2010,pv,auction,0.17,0.148,0.176


In [171]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
areas = base.mark_area(opacity=0.2).encode(
    y=alt.Y("p5:Q", sort=[]),
    y2=alt.Y2("p95:Q"),
    color=alt.Color(
        "tech:N",
        scale=alt.Scale(
            range=[
                colors["eco-orange"],
                colors["eco-mid-blue"],
                colors["eco-green"],
                scale_lightness(colors["eco-yellow"], 0.7),
            ]
        ),
    ),
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "avg:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                title="$/kWh",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=27,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".2f",
            ),
        ),
        color=alt.Color("tech:N", legend=None),
    )
    .transform_filter('datum.db=="lcoe"')
)
lines2 = (
    base.mark_point(strokeOpacity=0.8, opacity=0.6)
    .encode(
        y=alt.Y("avg:Q", sort=[]),
        fill=alt.Color(
            "tech:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-mid-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
        color=alt.Color(
            "tech:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-mid-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
    )
    .transform_filter('datum.db=="auction"')
)
title = alt.TitleParams(
    "Evolution of LCOE for renewable energy technologies",
    subtitle=["Levelised cost of electricity. Source: CCC, basedo IRENA (2020)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
fossil = (
    alt.Chart(
        pd.DataFrame(
            [{"x": 2008, "y": 0.050, "y2": 0.177}, {"x": 2024, "y": 0.050, "y2": 0.177}]
        )
    )
    .mark_area(fill=colors["eco-gray"], opacity=0.2)
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
ftext = (
    alt.Chart(pd.DataFrame([{"x": 2024, "y": 0.165, "t": "Fossil fuel range"}]))
    .mark_text(
        align="right", color=colors["eco-gray"], fontSize=10, dy=-7, dx=-5, angle=270
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2023, "y": 0.055, "t": "Circles:"},
                {"x": 2023.2, "y": 0.04, "t": "Auctions"},
                {"x": 2023, "y": 0.025, "t": "& PPAs"},
                {"x": 2018.5, "y": 0.31, "t": "Lines:"},
                {"x": 2018.5, "y": 0.295, "t": "LCOE"},
                {"x": 2018.7, "y": 0.28, "t": "database"},
            ]
        )
    )
    .mark_text(align="right", color=colors["eco-gray"], fontSize=10, dx=-5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2011, "y": 0.14, "t": "Offshore wind"},
                {"x": 2010.5, "y": 0.064, "t": "Onshore wind"},
                {"x": 2011, "y": 0.364, "t": "Concentrated solar"},
                {"x": 2012.7, "y": 0.21, "t": "Solar PV"},
            ]
        )
    )
    .mark_text(align="left", fontSize=10)
    .encode(
        x="x:Q",
        y="y:Q",
        text="t:N",
        color=alt.Color(
            "t:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-mid-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
    )
)
layer1 = (
    (
        (fossil + lines + lines2 + ftext + text1 + text2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [172]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
areas = base.mark_area(opacity=0.2).encode(
    y=alt.Y("p5:Q", sort=[]),
    y2=alt.Y2("p95:Q"),
    color=alt.Color(
        "tech:N",
        scale=alt.Scale(
            range=[
                colors["eco-orange"],
                colors["eco-light-blue"],
                colors["eco-green"],
                scale_lightness(colors["eco-yellow"], 0.7),
            ]
        ),
    ),
)
lines = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "avg:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="$/kWh",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=32,
                titleY=-3,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".2f",
            ),
        ),
        color=alt.Color("tech:N", legend=None),
    )
    .transform_filter('datum.db=="lcoe"')
)
lines2 = (
    base.mark_point(strokeOpacity=0.8, opacity=0.6)
    .encode(
        y=alt.Y("avg:Q", sort=[]),
        fill=alt.Color(
            "tech:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-light-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
        color=alt.Color(
            "tech:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-light-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
    )
    .transform_filter('datum.db=="auction"')
)
title = alt.TitleParams(
    "Evolution of LCOE for renewable energy technologies",
    subtitle=["Levelised cost of electricity. Source: CCC, basedo IRENA (2020)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
fossil = (
    alt.Chart(
        pd.DataFrame(
            [{"x": 2008, "y": 0.050, "y2": 0.177}, {"x": 2024, "y": 0.050, "y2": 0.177}]
        )
    )
    .mark_area(fill=colors["eco-gray"], opacity=0.2)
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
ftext = (
    alt.Chart(pd.DataFrame([{"x": 2024, "y": 0.165, "t": "Fossil fuel range"}]))
    .mark_text(
        align="right", color=colors["eco-gray"], fontSize=10, dy=-7, dx=-5, angle=270
    )
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2023, "y": 0.055, "t": "Circles:"},
                {"x": 2023.2, "y": 0.04, "t": "Auctions"},
                {"x": 2023, "y": 0.025, "t": "& PPAs"},
                {"x": 2018.5, "y": 0.31, "t": "Lines:"},
                {"x": 2018.5, "y": 0.295, "t": "LCOE"},
                {"x": 2018.7, "y": 0.28, "t": "database"},
            ]
        )
    )
    .mark_text(align="right", color=colors["eco-gray"], fontSize=10, dx=-5)
    .encode(x="x:Q", y="y:Q", text="t:N")
)
text2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2011, "y": 0.14, "t": "Offshore wind"},
                {"x": 2010.5, "y": 0.064, "t": "Onshore wind"},
                {"x": 2011, "y": 0.364, "t": "Concentrated solar"},
                {"x": 2012.7, "y": 0.21, "t": "Solar PV"},
            ]
        )
    )
    .mark_text(align="left", fontSize=10)
    .encode(
        x="x:Q",
        y="y:Q",
        text="t:N",
        color=alt.Color(
            "t:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-orange"],
                    colors["eco-light-blue"],
                    colors["eco-green"],
                    scale_lightness(colors["eco-yellow"], 0.7),
                ]
            ),
        ),
    )
)
layer1 = (
    (
        (fossil + lines + lines2 + ftext + text1 + text2).properties(
            height=300, width=400
        )
    )
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [173]:
# https://iea.blob.core.windows.net/assets/5e6b3821-bb8f-4df4-a88b-e891cd8251e3/WorldEnergyInvestment2021.pdf
# https://www.iea.org/data-and-statistics/charts/global-energy-supply-investment-by-sector-2019-2021-2
# https://iea.blob.core.windows.net/assets/a9da6027-f7c7-4aeb-9710-4f66906c59ab/WEI2021ForWEB.xlsx
df = pd.read_excel(
    "raw/WEI2021ForWEB.xlsx", sheet_name="1.2", skiprows=41, nrows=1, usecols="C:Z"
)
labels = [
    "Upstream",
    "Mid/downstream",
    "Coal supply",
    "Low-carbon fuels",
    "Renewable power",
    "Fossil fuel power",
    "Nuclear power",
    "Electricity networks and battery storage",
]
df.columns = [str(i) + "_" + str(j) for i in labels for j in [2019, 2020, "2021E"]]
df = df.T.reset_index()
df["industry"] = df["index"].str.split("_").str[0]
df["year"] = df["index"].str.split("_").str[1]
df.columns = ["index", "value", "industry", "year"]
df1 = df.set_index(["year", "industry"]).loc["2019"].reset_index()
df1["year"] = 2018
df1["value"] = 0
df = pd.concat([df1, df])
df1["year"] = 2017
df = pd.concat([df1, df])
df["index"] = df["industry"] + "_" + df["year"].astype(str)
df = (
    df.sort_values(["industry", "year"]).set_index("industry").loc[labels].reset_index()
)

In [174]:
f = "fig2_energy-investment"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,industry,index,value,year
0,Upstream,Upstream_2017,0.0,2017
1,Upstream,Upstream_2018,0.0,2018
2,Upstream,Upstream_2019,475.0,2019
3,Upstream,Upstream_2020,326.0,2020
4,Upstream,Upstream_2021E,351.0,2021E


In [175]:
base = alt.Chart(f2).encode(
    x=alt.X("index:N", sort=[], axis=None),
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="billion US$",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            labelFontSize=10,
            titleFontWeight="normal",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "industry:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-dark-blue"],
                colors["eco-purple"],
                scale_lightness(colors["eco-yellow"], 0.7),
                colors["eco-green"],
                colors["eco-mid-blue"],
                colors["eco-dot"],
                colors["eco-light-blue"],
                colors["eco-turquiose"],
            ]
        ),
    ),
)
bars = base.mark_bar(opacity=0.9).encode(x=alt.X("index:N", sort=[], axis=None))
axes = (
    alt.Chart(
        pd.DataFrame(
            [
                {"t": "Upstream", "y": 0, "x": "Upstream_2018"},
                {
                    "t": "Mid/downstream",
                    "y": 0,
                    "x": "Mid/downstream_2018",
                },
                {
                    "t": "Coal supply",
                    "y": 0,
                    "x": "Coal supply_2018",
                },
                {
                    "t": "Low-carbon fuels",
                    "y": 0,
                    "x": "Low-carbon fuels_2018",
                },
                {
                    "t": "Renewable power",
                    "y": 0,
                    "x": "Renewable power_2018",
                },
                {
                    "t": "Fossil fuel power",
                    "y": 0,
                    "x": "Fossil fuel power_2018",
                },
                {
                    "t": "Nuclear power",
                    "y": 0,
                    "x": "Nuclear power_2018",
                },
                {
                    "t": "Electricity networks and battery storage",
                    "y": 0,
                    "x": "Electricity networks and battery storage_2018",
                },
            ]
        )
    )
    .mark_text(angle=270, align="left")
    .encode(
        x=alt.X("x:N", sort=None, axis=None),
        y="y:Q",
        text="t:N",
        color=alt.Color("t:N", legend=None),
    )
)
year = (
    bars.mark_text(align="right", angle=60, dx=-8, dy=-3)
    .encode(text="year:N")
    .transform_filter("datum.industry=='Renewable power'")
    .transform_filter(
        alt.FieldOneOfPredicate(field="year", oneOf=["2019", "2020", "2021E"])
    )
)

title = alt.TitleParams(
    "Global energy supply investment by sector",
    subtitle=["Billion US$ 2019. Source: IEA (2021)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((bars + axes + year).properties(height=200, width=500))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [176]:
base = alt.Chart(f2).encode(
    x=alt.X("index:N", sort=[], axis=None),
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="billion US$",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    ),
    color=alt.Color(
        "industry:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-gray"],
                colors["eco-orange"],
                scale_lightness(colors["eco-yellow"], 0.7),
                colors["eco-green"],
                colors["eco-mid-blue"],
                colors["eco-dot"],
                colors["eco-light-blue"],
                colors["eco-turquiose"],
            ]
        ),
    ),
)
bars = base.mark_bar(opacity=0.9).encode(x=alt.X("index:N", sort=[], axis=None))
axes = (
    alt.Chart(
        pd.DataFrame(
            [
                {"t": "Upstream", "y": 0, "x": "Upstream_2018"},
                {
                    "t": "Mid/downstream",
                    "y": 0,
                    "x": "Mid/downstream_2018",
                },
                {
                    "t": "Coal supply",
                    "y": 0,
                    "x": "Coal supply_2018",
                },
                {
                    "t": "Low-carbon fuels",
                    "y": 0,
                    "x": "Low-carbon fuels_2018",
                },
                {
                    "t": "Renewable power",
                    "y": 0,
                    "x": "Renewable power_2018",
                },
                {
                    "t": "Fossil fuel power",
                    "y": 0,
                    "x": "Fossil fuel power_2018",
                },
                {
                    "t": "Nuclear power",
                    "y": 0,
                    "x": "Nuclear power_2018",
                },
                {
                    "t": "Electricity networks and battery storage",
                    "y": 0,
                    "x": "Electricity networks and battery storage_2018",
                },
            ]
        )
    )
    .mark_text(angle=270, align="left")
    .encode(
        x=alt.X("x:N", sort=None, axis=None),
        y="y:Q",
        text="t:N",
        color=alt.Color("t:N", legend=None),
    )
)
year = (
    bars.mark_text(align="right", angle=60, dx=-8, dy=-3)
    .encode(text="year:N")
    .transform_filter("datum.industry=='Renewable power'")
    .transform_filter(
        alt.FieldOneOfPredicate(field="year", oneOf=["2019", "2020", "2021E"])
    )
)

title = alt.TitleParams(
    "Global energy supply investment by sector",
    subtitle=["Billion US$ 2019. Source: IEA (2021)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((bars + axes + year).properties(height=200, width=500))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [177]:
# https://www.theccc.org.uk/publication/sixth-carbon-budget/
df = pd.read_excel(
    "raw/Copy of The-Sixth-Carbon-Budget-Charts-and-data-in-the-report.xlsx",
    sheet_name="Advice report Ch5&6",
    skiprows=64,
    nrows=12,
    usecols="G:AL",
)
df['Sector and metric']=df['Sector and metric'].str.replace('\(£b\)','').str.strip()
df = df.set_index("Sector and metric").stack().reset_index()
df.columns = ["sector", "year", "value"]

<ipython-input-177-57bba41248b9>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Sector and metric']=df['Sector and metric'].str.replace('\(£b\)','').str.strip()


In [178]:
f = "fig3_investment"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,sector,year,value
0,Surface transport CAPEX,2020,2.323158
1,Surface transport CAPEX,2021,3.482380
2,Surface transport CAPEX,2022,4.764758
3,Surface transport CAPEX,2023,6.269741
4,Surface transport CAPEX,2024,8.072776


In [179]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area1 = base.mark_area(opacity=0.7).encode(
    y=alt.Y(
        "value:Q",
        stack=True,
        axis=alt.Axis(
            grid=True,
            title="£ billion / year",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    ),
    color=alt.Color('sector:N',sort=[],legend=None,scale=alt.Scale(range=category_color))
).transform_filter('datum.sector!="Total"')
texts=area1.mark_text(align='left',dx=5).encode(text='sector:N').transform_filter('datum.year==2050')
title = alt.TitleParams(
    "Capital investment costs and operating cost savings",
    subtitle=["The Sixth Carbon Budget - Balanced Pathway. Source: CCC (2020)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1+texts))
    .properties(height=300, width=400)
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [180]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=12,
            labelFontSize=12,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
area1 = base.mark_area(opacity=0.7).encode(
    y=alt.Y(
        "value:Q",
        stack=True,
        axis=alt.Axis(
            grid=False,
            title="£ billion / year",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    ),
    color=alt.Color('sector:N',sort=[],legend=None,scale=alt.Scale(range=category_color))
).transform_filter('datum.sector!="Total"')
texts=area1.mark_text(align='left',dx=5).encode(text='sector:N').transform_filter('datum.year==2050')
title = alt.TitleParams(
    "Capital investment costs and operating cost savings",
    subtitle=["The Sixth Carbon Budget - Balanced Pathway. Source: CCC (2020)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((area1+texts))
    .properties(height=300, width=400)
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png")
readme=re.sub(f, f+'_dark', readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)